# Градиентный бустинг

## Теория

Градиентный бустинг — это алгоритм машинного обучения, который строит ансамбль неглубоких деревьев решений (глубина 2-8), путем **направленного** обучения этих базовых алгоритмов (последующие учатся на ошибках предыдущих).

Бустинг - **сумма базовых алгоритмов**, а не их усреднение:

$$a_N(x) = \sum\limits_{i=1}^N b_i(x)$$

Как это работает? Пусть, например, мы решаем задачу регрессии и используем в качестве функции потерь среднеквадратическую ошибку: $MSE(a,X) = \frac{1}{l} \sum\limits_{i=1}^l(a(x_i)-y_i)^2$

Построим $N$ деревьев, исходя из соображения, что именно такое количество деревьев необходимо для минимизации ошибки. Тогда:

$a_1(x) = b_1(x) = argmin_b \frac{1}{l} \sum\limits_{i=1}^l(b(x_i)-y_i)^2$

$a_2(x) = b_1(x) + b_2(x) = argmin_b \frac{1}{l} \sum\limits_{i=1}^l (b_1(x_i) + b(x_i) - y_i)^2 = argmin_b \frac{1}{l} \sum\limits_{i=1}^l (b(x_i) - (y_i - b_1(x_i)))^2$

$...$

$a_N(x) = argmin_b \frac{1}{l} \sum\limits_{i=1}^l (b(x_i) - (y_i - \sum\limits_{n=1}^{N-1} b_n(x_i)))^2$

### Несколько крутых визуализаций
1. [Gradient Boosting explained [demonstration]](http://arogozhnikov.github.io/2016/06/24/gradient_boosting_explained.html)
2. [Gradient Boosting Interactive Playground](http://arogozhnikov.github.io/2016/07/05/gradient_boosting_playground.html)
3. Каноничная картинка с гольфом:
<img src="pictures/golf_mse.png" width=400 height=400 />

## Описание алгоритма градиентного бустинга

### Дано
* Обучающая выборка: ${(x_i, s_i)}_{i=1}^l$
* Дифференцируемая функция потерь: $L(y, z)$, где $y$ - истинный ответ, $z$ - прогноз алгоритма на некотором объекте
    * В задаче регрессии: среднеквадратическая ошибка - $L(y,z) = (z-y)^2$
    * В задаче классификации: логистическая функция потерь - $L(y,z) = log(1+exp(-yz))$
* Некоторый базовый алгоритм (решающее дерево): $b_i(x)$

### Инициализация
    
Прежде всего необходимо построить первый базовый алгоритм $b_0(x)$, он должен быть достаточно простым, например:
* В задаче регрессии:
    * $b_0(x) = 0$
    * $b_0(x) = \frac{1}{l} \sum\limits_{i=1}^l y_i$ - среднее по выборке
* В задаче классификации:
    * $b_0(x) = argmax_{y \in Y} \sum\limits_{i=1}^l [y_i = y]$ - метка самого распространенного класса в выборке

### Обучение
    
Пусть было построено $N-1$ базовых алгоритмов: $a_{N-1}(x) = \sum\limits_{n=1}^{N-1} b_n(x)$

Добавим в композицию алгоритм $b_N(x)$, для которого должно выполняться: $\sum\limits_{i=1}^l L(y_i, a_{N-1}(x_i) + b(x_i)) \rightarrow \min\limits_{b}$

Введем **вектор сдвигов** $s = (s_1, ..., s_l) = (b_n(x_1), ..., b_n(x_l))$. Компоненты этого вектора - значения, которые должен принимать алгоритм $b_N(x)$ на объектах выборки, чтобы ошибка композиции была минимальной:

$$F(s) = \sum\limits_{i=1}^l L(y_i, a_{N-1}(x_i) + s_i) \rightarrow \min\limits_{s}$$

По сути, мы решаем задачу оптимизации, в которой ищем такой ветор $s$, который будет минимизировать функцию $F(s)$. Таким образом, на каждой итерации градиентного бустинга вычисляется антиградиент функции ошибки композиции алгоритмов:

$$
\begin{equation*}
s = -\nabla F = \left(\begin{array}{cccc}
-L_z'(y_1, a_{N-1}(x_1)) \\ \ldots \\ -L_z'(y_l, a_{N-1}(x_l))
\end{array} \right) 
\end{equation*}
$$

Таким образом, в композицию мы добавляем алгоритм $b_n(x) = argmin_b \frac{1}{l} \sum\limits_{i=1}^l L(b(x_i), s_i)$.

Обучение продолжается до тех пор, пока не выполнен **критерий останова**.

## Борьба с переобучением

Есть несколько основных способов борьбы с переобучением в градиентном бустинге:
* **Сокращение размера шага**:
    * Вводится гиперпараметр $\eta \in (0, 1]$ — длина шага: $a_N(x) = a_{N-1}(x) + \eta b_N(x)$
* **Увеличение количества деревьев** $N$
* **Обучение на подвыборке объектов (bagging / стохастический градиентный спуск)**

## Литература

1. Специализация "Машинное обучение и анализ данных" от МФТИ и Яндекса на Coursera
2. [Дьяконов - Градиентный бустинг](https://alexanderdyakonov.files.wordpress.com/2017/06/book_boosting_pdf.pdf)
3. [Соколов - XGboost](https://github.com/esokolov/ml-course-hse/blob/master/2018-fall/lecture-notes/lecture10-ensembles.pdf)
4. [ИТМО - XGBoost](http://neerc.ifmo.ru/wiki/index.php?title=XGBoost&mobileaction=toggle_view_desktop#:~:text=%D0%93%D1%80%D0%B0%D0%B4%D0%B8%D0%B5%D0%BD%D1%82%D0%BD%D1%8B%D0%B9%20%D0%B1%D1%83%D1%81%D1%82%D0%B8%D0%BD%D0%B3%20%E2%80%94%20%D1%8D%D1%82%D0%BE%20%D1%82%D0%B5%D1%85%D0%BD%D0%B8%D0%BA%D0%B0%20%D0%BC%D0%B0%D1%88%D0%B8%D0%BD%D0%BD%D0%BE%D0%B3%D0%BE,%D0%B2%20%D0%BE%D1%82%D0%BB%D0%B8%D1%87%D0%B8%D0%B5%2C%20%D0%BD%D0%B0%D0%BF%D1%80%D0%B8%D0%BC%D0%B5%D1%80%20%D0%BE%D1%82%20%D0%B1%D1%8D%D0%B3%D0%B3%D0%B8%D0%BD%D0%B3%D0%B0.)
5. [Открытый курс машинного обучения. Тема 10. Градиентный бустинг](https://habr.com/ru/company/ods/blog/327250/#1-vvedenie-i-istoriya-poyavleniya-bustinga)
6. [CatBoost, XGBoost и выразительная способность решающих деревьев](https://habr.com/ru/company/ods/blog/645887/)

# CatBoost vs. LightGBM vs. XGBoost

### [XGBoost (eXtreme Gradient Boosting)](https://arxiv.org/abs/1603.02754)

* Релиз: 2014
* Асимметричные деревья (**разбиение вглубь - параметр max_depth**)
* Распределенное обучение
* Предварительная сортировка значений для вычисления наилучшего разбиения (pre-sorted algorithm & histogram-based algorithm)
    * Значения каждого признака сортируются
    * Значения исходного признака скалируются и поиск наилучшего разбиения проходит не по исходному значению признака, а по полученным перцентилям
    * Наилучшее разбиение выбирается по перцентилям среди всех признаков по критерию информативности
* Оптимизированная работа с кэшем
* Эффективная работа с пропущенными значениями
* Гиперпараметры для тюнинга:
    * n_estimators
    * max_depth
    * min_child_weight

### [LightGBM (Light Gradient Boosted Machine)](https://papers.nips.cc/paper/2017/hash/6449f44a102fde848669bdd9eb6b76fa-Abstract.html)

* Релиз: 2016
* Асимметричные деревья (**разбиение вширь - параметр num_leaves**)
* Поиск наилучшего разбиения: Gradient-based One-Side Sampling (GOSS)
* Условно может работать с категориальными признаками (параметр categorical_feature)
* Гиперпараметры для тюнинга:
    * num_leaves
    * min_data_in_leaf
    * max_depth
    * max_bin

### [CatBoost (Category Boosting)](https://arxiv.org/abs/1810.11363)

* Релиз: 2017
* Несмещённый упорядоченный бустинг
* По умолчанию строятся симметричные (или сбалансированные) деревья (параметр по умолчанию grow_policy='SymmetricTree')
* Жадный способ построения деревьев
* Упорядоченный бустинг (параметр boosting_type='Ordered') для датасетов небольшого размера
* Может работать с категориальными признаками при помощи упорядоченного target-кодирования (параметр one_hot_max_size)
* Разделение ветвей не только по отдельным признакам, но и по их комбинациям (параметр max_ctr_complexity)
* Может работать с тектовыми данными с помощью bag-of-words (параметр text_features метода fit)
* Возможность обучения на GPU (параметр task_type='GPU' в методе fit)
* Гиперпараметры для тюнинга:
    * iterations
    * depth
    * min_data_in_leaf

### Литература
1. [Medium - CatBoost vs. Light GBM vs. XGBoost](https://towardsdatascience.com/catboost-vs-light-gbm-vs-xgboost-5f93620723db)
2. [Medium - CatBoost vs. LightGBM vs. XGBoost](https://towardsdatascience.com/catboost-vs-lightgbm-vs-xgboost-c80f40662924)
3. [CatBoost, XGBoost и выразительная способность решающих деревьев](https://habr.com/ru/company/ods/blog/645887/)

### Примеры

In [1]:
import time
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import preprocessing

warnings.filterwarnings('ignore')

In [2]:
train_df = pd.read_csv('tmp/train.csv')
X_train = train_df.drop(columns=['Survived'])
y_train = train_df['Survived']

In [3]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
text_features = ['Name', 'Ticket', 'Cabin']

X_train = X_train.drop(columns=text_features)

In [5]:
cat_features = ['Sex', 'Embarked']

for feat in cat_features:
    X_train[feat] = X_train[feat].astype('category')

### XGBoost

In [6]:
import xgboost as xgb

In [7]:
model = xgb.XGBClassifier()

param_dist = {"max_depth": [10, 30, 50],
              "min_child_weight" : [1, 3, 6],
              "n_estimators": [200],
              "learning_rate": [0.05, 0.1, 0.16]}

grid_search = GridSearchCV(model,
                           param_grid=param_dist,
                           cv=3,
                           verbose=10,
                           n_jobs=-1)

grid_search.fit(X_train.drop(columns=cat_features), y_train)
grid_search.best_estimator_

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:    8.4s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    9.8s
[Parallel(n_jobs=-1)]: Done  75 out of  81 | elapsed:   12.5s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed:   13.1s finished


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.05, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=10, max_leaves=0, min_child_weight=6,
              missing=nan, monotone_constraints='()', n_estimators=200,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [8]:
model = grid_search.best_estimator_
model.fit(X_train.drop(columns=cat_features), y_train)
train_proba = model.predict_proba(X_train.drop(columns=cat_features))
metrics.roc_auc_score(y_train, train_proba[:, 1])

0.9583586318558996

### Light GBM

In [9]:
import lightgbm as lgb

In [10]:
lg = lgb.LGBMClassifier(silent=False)

param_dist = {"max_depth": [25, 50, 75],
              "learning_rate" : [0.01, 0.05, 0.1],
              "num_leaves": [300, 900, 1200],
              "n_estimators": [200]}

grid_search = GridSearchCV(lg,
                           param_grid=param_dist,
                           n_jobs=-1,
                           cv=3,
                           scoring="roc_auc",
                           verbose=10)

grid_search.fit(X_train, y_train)
grid_search.best_estimator_

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done  75 out of  81 | elapsed:    3.3s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed:    3.5s finished


LGBMClassifier(learning_rate=0.01, max_depth=25, n_estimators=200,
               num_leaves=300, silent=False)

In [11]:
params = {"max_depth": 250,
          "learning_rate": 0.01,
          "num_leaves": 900,  
          "n_estimators": 300}

In [12]:
d_train = lgb.Dataset(X_train.drop(columns=cat_features),
                      label=y_train,
                      free_raw_data=False)

# Without Categorical Features
model = lgb.train(params, d_train)
predict = model.predict(X_train.drop(columns=cat_features))
metrics.roc_auc_score(y_train, predict)

0.9427640899455683

In [13]:
d_train = lgb.Dataset(X_train,
                      label=y_train,
                      categorical_feature=cat_features,
                      free_raw_data=False)

# With Categorical Features
model = lgb.train(params, d_train)
predict = model.predict(X_train)
metrics.roc_auc_score(y_train, predict)

0.9678389203123169

### CatBoost

In [14]:
import catboost as cb

In [15]:
cate_features_index = np.where(X_train.dtypes != float)[0]

for feat in cat_features:
    X_train[feat] = X_train[feat].astype(str)

In [20]:
params = {'depth': [4, 7, 10],
          'learning_rate' : [0.03, 0.1, 0.15],
          'l2_leaf_reg': [1, 4, 9],
          'iterations': [30]}

model = cb.CatBoostClassifier(eval_metric='Accuracy', random_seed=42)

train_pool = cb.Pool(X_train, y_train, cat_features=cat_features)

grid_search_results = model.grid_search(params, train_pool, shuffle=False)

In [18]:
# With Categorical features
clf = cb.CatBoostClassifier(eval_metric="AUC",
                            depth=10, 
                            iterations=50,
                            l2_leaf_reg=9, 
                            learning_rate=0.15)

clf.fit(X_train, y_train, cat_features=cate_features_index)
predict = clf.predict(X_train)
metrics.roc_auc_score(y_train, predict)

0:	total: 489us	remaining: 24ms
1:	total: 3.36ms	remaining: 80.7ms
2:	total: 4.76ms	remaining: 74.7ms
3:	total: 5.51ms	remaining: 63.3ms
4:	total: 5.97ms	remaining: 53.8ms
5:	total: 6.5ms	remaining: 47.7ms
6:	total: 7.16ms	remaining: 44ms
7:	total: 8.07ms	remaining: 42.4ms
8:	total: 12.6ms	remaining: 57.4ms
9:	total: 16.9ms	remaining: 67.4ms
10:	total: 17.4ms	remaining: 61.5ms
11:	total: 22.1ms	remaining: 69.9ms
12:	total: 26.8ms	remaining: 76.4ms
13:	total: 29.9ms	remaining: 76.9ms
14:	total: 31.9ms	remaining: 74.5ms
15:	total: 37.1ms	remaining: 78.8ms
16:	total: 43.2ms	remaining: 83.9ms
17:	total: 48.2ms	remaining: 85.6ms
18:	total: 48.8ms	remaining: 79.7ms
19:	total: 54.2ms	remaining: 81.3ms
20:	total: 65.4ms	remaining: 90.4ms
21:	total: 81.4ms	remaining: 104ms
22:	total: 87.4ms	remaining: 103ms
23:	total: 93.5ms	remaining: 101ms
24:	total: 99ms	remaining: 99ms
25:	total: 99.6ms	remaining: 91.9ms
26:	total: 103ms	remaining: 87.6ms
27:	total: 110ms	remaining: 86.3ms
28:	total: 118ms	

0.8027274470328828

In [19]:
# With Categorical features
clf = cb.CatBoostClassifier(eval_metric="AUC",
                            one_hot_max_size=31,
                            depth=10,
                            iterations=50,
                            l2_leaf_reg=9, 
                            learning_rate=0.15)

clf.fit(X_train, y_train, cat_features=cate_features_index)
predict = clf.predict(X_train)
metrics.roc_auc_score(y_train, predict)

0:	total: 1.84ms	remaining: 90ms
1:	total: 2.63ms	remaining: 63.1ms
2:	total: 3.37ms	remaining: 52.8ms
3:	total: 8.52ms	remaining: 98ms
4:	total: 9.2ms	remaining: 82.8ms
5:	total: 13.9ms	remaining: 102ms
6:	total: 19.1ms	remaining: 117ms
7:	total: 21.3ms	remaining: 112ms
8:	total: 26.3ms	remaining: 120ms
9:	total: 31.3ms	remaining: 125ms
10:	total: 33.2ms	remaining: 118ms
11:	total: 37.6ms	remaining: 119ms
12:	total: 40.4ms	remaining: 115ms
13:	total: 41.2ms	remaining: 106ms
14:	total: 42ms	remaining: 98.1ms
15:	total: 42.7ms	remaining: 90.8ms
16:	total: 46.9ms	remaining: 91.1ms
17:	total: 47.9ms	remaining: 85.1ms
18:	total: 48.3ms	remaining: 78.8ms
19:	total: 52.5ms	remaining: 78.7ms
20:	total: 56.6ms	remaining: 78.1ms
21:	total: 57.3ms	remaining: 72.9ms
22:	total: 61.6ms	remaining: 72.4ms
23:	total: 66.7ms	remaining: 72.3ms
24:	total: 70.6ms	remaining: 70.6ms
25:	total: 71ms	remaining: 65.6ms
26:	total: 74.6ms	remaining: 63.6ms
27:	total: 78.2ms	remaining: 61.4ms
28:	total: 81.9ms	re

0.8314159716230467